# 5. Obspy

Obspy is the worldwide library for seismic data manipulation in Python. It includes nearly everything you need to work properly at the institute. 

If you want to search for something into Obspy, the website is really well documented [ObsPy Documentation](http://docs.obspy.org/) and you will find for sure the love of your life on it.

Lesson based on [MESS 2014](https://github.com/obspy/mess2014-notebooks/) and [Seismo-Live](https://krischer.github.io/seismo_live_build/tree/index.html)

The core functionality of ObsPy is provided by..

- ..the most important base classes..
  * the **`UTCDateTime`** class handles time information.
  * the **`Stream`**/**`Trace`** classes handle waveform data.
  * the **`Catalog`**/**`Event`**/... classes handle event metadata (modelled after QuakeML).
  * the **`Inventory`**/**`Station`**/**`Response`**/... classes handle station metadata (modelled after FDSN StationXML).


- ..and the associated functions:
  * The **`read`** function. Reads all kinds of waveform file formats. Outputs a **`Stream`** object.
  * The **`read_events`** function. Reads QuakeML (and MCHEDR) files. Outputs a **`Catalog`** object.
  * The **`read_inventory`** function. Reads FDSN StationXML files. Outputs an **`Inventory`** object.


- the most important classes/functions can be imported from main namespace (`from obspy import ...`)
- Unified interface and functionality for handling waveform data regardless of data source
- **`read`**, **`read_events`** and **`read_inventory`** functions access the appropriate file-format submodule/plugin using filetype autodiscovery
- `obspy.core.util` includes some generally useful utility classes/functions (e.g. for geodetic calculations, Flinn-Engdahl regions, ..)
- some convenience command line scripts are also included (e.g. `obspy-plot`, `obspy-print`, `obspy-scan`, ..)


## 5.1. Handling Timestamps and File Format

### Handling Timestamps

This is a bit dry but not very difficult and important to know. It is used everywhere in ObsPy!


* All absolute time values are consistently handled with this class
* Based on a double precision POSIX timestamp for accuracy
* Timezone can be specified at initialization (if necessary)
* In Coordinated Universal Time (UTC) so no need to deal with timezones, daylight savings, ...

In [ ]:
from obspy import UTCDateTime

print(UTCDateTime("2011-03-11T05:46:23.2"))        # mostly time strings defined by ISO standard
print(UTCDateTime("2011-03-11T14:46:23.2+09:00"))  # non-UTC timezone input
print(UTCDateTime(2011, 3, 11, 5, 46, 23, 200000))
print(UTCDateTime(1299822383.2))

In [ ]:
# Current time can be initialized by leaving out any arguments
print(UTCDateTime())

If you want to access to the attributes of time :

In [ ]:
time = UTCDateTime()
print(time.year)
print(time.julday)
print(time.timestamp)
print(time.weekday)

If you want to now about all the attributes, you can press tab in the below cell

In [ ]:
time.

To do some operations with time, you have to use the second as the quantity. You can __add or substract__ seconds from one ``UTCDatetime`` object __BUT__ you can only do the __difference__ between two `UTCDateTime` objects.

The result of the first one will be a __delta__ in seconds and the other will be an ``UTCDateTime`` object.

In [ ]:
time = UTCDateTime()
print(time)

In [ ]:
time2 = UTCDateTime()
print(time2)

In [ ]:
print('time + 1 hour:', time + 3600) # one hour
print('time - 1 hour:', time - 3600)
print('Delta between time2 and time:', time2 - time, 'seconds')

In [ ]:
from time import sleep

t1 = UTCDateTime()
sleep(5) # stop the script for 5 seconds
t2 = UTCDateTime()

print('t1:', t1, 'and t2:', t2)
print('t1 + 1 hour:', t1 + 3600) # one hour
print('t1 - 1 hour:', t1 - 3600)
print('Delta between t2 and t1:', t2 - t1, 'seconds')

### Handling File Format

#### SEED Identifiers

According to the  [SEED standard](www.fdsn.org/seed_manual/SEEDManual_V2.4.pdf), which is fairly well adopted, the following nomenclature is used to identify seismic receivers:

* **Network code**: Identifies the network/owner of the data. Assigned by the FDSN and thus unique.
* **Station code**: The station within a network. *NOT UNIQUE IN PRACTICE!* Always use together with a network code!
* **Location ID**: Identifies different data streams within one station. Commonly used to logically separate multiple instruments at a single station.
* **Channel codes**: Three character code: 1) Band and approximate sampling rate, 2) The type of instrument, 3) The orientation

This results in full ids of the form **NET.STA.LOC.CHAN**, e.g. **RD.SONA0..SHZ**.

In seismology we generally distinguish between three separate types of data:

1. **Waveform Data** - The actual waveforms as time series.
2. **Station Data** - Information about the stations' operators, geographical locations, and the instrument's responses.
3. **Event Data** - Information about earthquakes.

Some formats have elements of two or more of these.

#### Waveform Data

![stream](images/Stream_Trace.svg)

There are a myriad of waveform data formats but in Europe and the USA two formats dominate: **MiniSEED** and **SAC**


##### MiniSEED

* This is what you get from datacenters and also what they store, thus the original data
* Can store integers and single/double precision floats
* Integer data (e.g. counts from a digitizer) are heavily compressed: a factor of 3-5 depending on the data
* Can deal with gaps and overlaps
* Multiple components per file
* Contains only the really necessary parameters and some information for the data providers

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

In [ ]:
from obspy import read

# ObsPy automatically detects the file format.
st = read("data/example.mseed") # STREAM
print(st)

tr = st[0] # TRACE

# Fileformat specific information is stored here.
print(tr.stats)

In [ ]:
st.plot()

In [ ]:
# This is a quick interlude to teach you basics about how to work
# with Stream/Trace objects.

# We need to copy because some actions are modifying the original data
st2 = st.copy()

# To use only part of a Stream, use the select() function.
st2 = st2.select(component="Z")
print(st2)

# Stream objects behave like a list of Trace objects.
tr = st2[0]
print(tr.stats)
tr.plot()

# Some basic processing. Please note that these modify the
# existing object.
tr.detrend("linear")
tr.taper(type="hann", max_percentage=0.05)
tr.filter("lowpass", freq=0.5)

tr.plot()

In [ ]:
# You can write it again by simply specifing the format.
st.write("temp.mseed", format="mseed")

##### SAC

* Custom format of the `sac` code.
* Simple header and single precision floating point data.
* Only a single component per file and no concept of gaps/overlaps.
* Used a lot due to `sac` being very popular and the additional basic information that can be stored in the header.

In [ ]:
st = obspy.read("data/example.sac") + obspy.read("data/waveform_BFO_BHE.sac")
print(st)
print(st[0].stats)

#### Station Data

![inv](images/Inventory.svg)

Station data contains information about the organziation that collections the data, geographical information, as well as the instrument response. It mainly comes in three formats:

* `(dataless) SEED`: Very complete but pretty complex and binary. Still used a lot, e.g. for the Arclink protocol
* `RESP`: A strict subset of SEED. ASCII based. Contains **ONLY** the response.
* `StationXML`: Essentially like SEED but cleaner and based on XML. Most modern format and what the datacenters nowadays serve. **Use this if you can.**


ObsPy can work with all of them but today we will focus on StationXML.

They are XML files:

In [ ]:
from obspy import read_inventory

# Use the read_inventory function to open them.
inv = read_inventory("data/all_stations.xml")
print(inv)

You can see that they can contain an arbirary number of networks, stations, and channels.

In [ ]:
# As well as a plot the instrument response.
inv.select(network="IV", station="SALO", channel="BHZ").plot_response(0.001)

In [ ]:
# Coordinates of single channels can also be extraced. This function
# also takes a datetime arguments to extract information at different
# points in time.
inv.get_coordinates("IV.SALO..BHZ")

In [ ]:
# And it can naturally be written again, also in modified state.
inv.select(channel="BHZ").write("data/temp.xml", format="stationxml")

#### Event Data

![events](./images/Event.svg)

Event data is essentially served in either very simple formats like NDK or the CMTSOLUTION format used by many waveform solvers:

Datacenters on the hand offer QuakeML files, which are surprisingly complex in structure but can store complex relations.

In [ ]:
# Read QuakeML files with the read_events() function.
cat = obspy.read_events("data/GCMT_2014_04_01__Mw_8_1.xml") + obspy.read_events("data/event_tohoku_mainshock.xml")
print(cat)

In [ ]:
print(cat[0])

In [ ]:
# Once again they can be written with the write() function.
cat.write("data/temp_quake.xml", format="quakeml")

To show off some more things, I added a file containing all events from 2014 in the GCMT catalog.

In [ ]:
cat = obspy.read_events("data/2014.ndk")

print(cat)

In [ ]:
cat.plot() # doesn't work here, i don't have a map software on my server

In [ ]:
cat.filter("magnitude > 7")

### Handling Waveform Data

In [ ]:
from obspy import read
st = read("./data/waveform_PFO.mseed", format="mseed")
print(st)

- UNIX wildcards can be used to read multiple files simultaneously
- automatic file format detection, no need to worry about file formats

  - currently supported: **mseed, sac, segy, seg2, gse1/2, seisan, sh, datamark, css, wav, y, Q (keeps growing...)**
  - more file formats are included whenever a basic reading routine is provided (or e.g. sufficient documentation on data compression etc.)
  - custom user-specific file formats can be added (through plugin) to filetype autodiscovery in local ObsPy installation by user

In [ ]:
from obspy import read
st = read("./data/waveform_*")
print(st)

- for MiniSEED files, only reading short portions of files without all of the file getting read into memory is supported (saves time and memory when working on large collections of big files)

In [ ]:
from obspy import UTCDateTime

t = UTCDateTime("2011-03-11T05:46:23.015400Z")
st = read("./data/waveform_*", starttime=t + 10 * 60, endtime=t + 12 * 60)
print(st)
st[0].plot()

#### The Stream Object

 - A Stream object is a collection of Trace objects

In [ ]:
from obspy import read
st = read("./data/waveform_PFO.mseed")
print(type(st))

In [ ]:
print(st)

In [ ]:
print(st.traces)

In [ ]:
print(st[0])

- More traces can be assembled using **`+`** operator (or using the `.append()` and `.extend()` methods)

In [ ]:
st1 = read("./data/waveform_PFO.mseed")
st2 = read("./data/waveform_PFO_synthetics.mseed")

st = st1 + st2
print(len(st), 'Traces')

st3 = read("./data/waveform_BFO_BHE.sac")

st += st3
print(len(st), 'Traces')

 - convenient (and nicely readable) looping over traces

In [ ]:
for tr in st:
    print(tr.id)

 - Stream is useful for applying the same processing to a larger number of different waveforms or to group Traces for processing (e.g. three components of one station in one Stream)

#### The Trace Object

- a Trace object is a single, contiguous waveform data block (i.e. regularly spaced time series, no gaps)
- a Trace object contains a limited amount of metadata in a dictionary-like object (as **`Trace.stats`**) that fully describes the time series by specifying..
  * recording location/instrument (network, station, location and channel code)
  * start time
  * sampling rate

In [ ]:
st = read("./data/waveform_PFO.mseed")
tr = st[0]  # get the first Trace in the Stream
print(tr)

In [ ]:
print(tr.stats)

- For custom applications it is sometimes necessary to directly manipulate the metadata of a Trace.
- The metadata of the Trace will **stay consistent**, as all values are derived from the starttime, the data and the sampling rate and are **updated automatically**

In [ ]:
print(tr.stats.delta, "|", tr.stats.endtime)

In [ ]:
tr.stats.sampling_rate = 5.0
print(tr.stats.delta, "|", tr.stats.endtime)

In [ ]:
print(tr.stats.npts, "|", tr.stats.endtime)

In [ ]:
tr.data = tr.data[:100]
print(tr.stats.npts, "|", tr.stats.endtime)

- convenience methods make basic manipulations simple

In [ ]:
tr = read("./data/waveform_PFO.mseed")[0]
tr.plot()

In [ ]:
print(tr)
tr.resample(sampling_rate=100.0)
print(tr)

In [ ]:
print(tr)
tr.trim(tr.stats.starttime + 12 * 60, tr.stats.starttime + 14 * 60)
print(tr)
tr.plot()

In [ ]:
tr.detrend("linear")
tr.taper(max_percentage=0.05, type='cosine')
tr.filter("lowpass", freq=0.1)
tr.plot()

In [ ]:
# try tr.<Tab> for other methods defined for Trace
tr.detrend?

- Raw data available as a [**`numpy.ndarray`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (as **`Trace.data`**)

In [ ]:
print(tr.data[:20])

- Data can be directly modified e.g. ..

..by doing arithmetic operations (fast, handled in C by NumPy)

In [ ]:
print(tr.data ** 2 + 0.5)

..by using [**`numpy.ndarray`** builtin methods](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (also done in C by NumPy)

In [ ]:
print(tr.data.max())
print(tr.data.mean())
print(tr.data.ptp())
# try tr.data.<Tab> for a list of numpy methods defined on ndarray

..by using **`numpy`** functions (also done in C by NumPy)

In [ ]:
import numpy as np
print(np.abs(tr.data))
# you can try np.<Tab> but there is a lot in there
# try np.a<Tab>

..by feeding pointers to existing C/Fortran routines from inside Python!

This is done internally in several places, e.g. for cross correlations, beamforming or in third-party filetype libraries like e.g. libmseed.

- Trace objects can also be manually generated with data in a [**`numpy.ndarray`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) (e.g. when needing to parse waveforms from non-standard ascii files)

In [ ]:
from obspy import Trace
import numpy as np

x = np.random.randint(-100, 100, 500)
tr = Trace(data=x)
tr.stats.station = "XYZ"
tr.stats.starttime = UTCDateTime()

tr.plot()

- Stream objects can be assembled from manually generated Traces

In [ ]:
from obspy import Stream

tr2 = Trace(data=np.random.randint(-300, 100, 1000))
tr2.stats.starttime = UTCDateTime()
tr2.stats.sampling_rate = 10.0
st = Stream([tr, tr2])

st.plot()

#### Builtin methods defined on **`Stream`** / **`Trace`**

- Most methods that work on a Trace object also work on a Stream object. They are simply executed for every trace. [See ObsPy documentation for an overview of available methods](http://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html) (or try **`st.<Tab>`**).
 - **`st.filter()`** - Filter all attached traces.
 - **`st.trim()`** - Cut all traces.
 - **`st.resample()`** / **`st.decimate()`** - Change the sampling rate.
 - **`st.trigger()`** - Run triggering algorithms.
 - **`st.plot()`** / **`st.spectrogram()`** - Visualize the data.
 - **`st.attach_response()`**/**`st.remove_response()`**, **`st.simulate()`** - Instrument correction
 - **`st.merge()`**, **`st.normalize()`**, **`st.detrend()`**, **`st.taper()`**, ...
- A **`Stream`** object can also be exported to many formats, so ObsPy can be used to convert between different file formats.

In [ ]:
st = read("./data/waveform_*.sac")
st.write("output_file.mseed", format="MSEED")

### Exercises about Timestamps

Calculate the number of days passed since the Tohoku main shock (`2011-03-11T05:46:23.2`).

In [ ]:
from obspy import UTCDateTime


Make a list of 10 UTCDateTime objects, starting today at 10:00 with a spacing of 90 minutes.

In [ ]:
times = []


Below is a list of strings with origin times of magnitude 8+ earthquakes since 2000 (fetched from IRIS). Assemble a list of inter-event times in days. Use matplotlib to display a histogram.

In [ ]:
times = ["2001-06-23T20:33:09",
         "2003-09-25T19:50:07",
         "2004-12-23T14:59:00",
         "2004-12-26T00:58:52",
         "2005-03-28T16:09:35",
         "2006-06-01T18:57:02",
         "2006-06-05T00:50:31",
         "2006-11-15T11:14:14",
         "2007-01-13T04:23:23",
         "2007-04-01T20:39:56",
         "2007-08-15T23:40:58",
         "2007-09-12T11:10:26",
         "2009-09-29T17:48:11",
         "2010-02-27T06:34:13",
         "2011-03-11T05:46:23",
         "2012-04-11T08:38:36",
         "2012-04-11T10:43:10",
         "2013-05-24T05:44:48"]

In [ ]:
import matplotlib.pyplot as plt



### Trace Exercises
 - Make an **`numpy.ndarray`** with zeros and (e.g. use **`numpy.zeros()`**) and put an ideal pulse somewhere in it
 - initialize a **`Trace`** object with your data array
 - Fill in some station information (e.g. network, station, ..)
 - Print trace summary and plot the trace
 - Change the sampling rate to 20 Hz
 - Change the starttime of the trace to the start time of this session
 - Print the trace summary and plot the trace again

In [ ]:
import numpy as np
from obspy import Trace, UTCDateTime




- Use **`tr.filter(...)`** and apply a lowpass filter with a corner frequency of 1 Hertz.
- Display the preview plot, there are a few seconds of zeros that we can cut off.

- Use **`tr.trim(...)`** to remove some of the zeros at start and at the end
- show the preview plot again

- Scale up the amplitudes of the trace by a factor of 500
- Add standard normal gaussian noise to the trace (use [**`np.random.randn()`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html))
- Display the preview plot again

### Stream Exercises

- Read all Tohoku example earthquake data into a stream object ("./data/waveform\_\*")
- Print the stream summary

In [ ]:
from obspy import read



- Use **`st.select()`** to only keep traces of station BFO in the stream. Show the preview plot.

- trim the data to a 10 minute time window around the first arrival (just roughly looking at the preview plot)
- display the preview plot and spectrograms for the stream (with logarithmic frequency scale, use `wlen=50` for the spectrogram plot)

- remove the linear trend from the data, apply a tapering and a lowpass at 0.1 Hertz
- show the preview plot again

### Correction about Timestamps

Calculate the number of days passed since the Tohoku main shock (`2011-03-11T05:46:23.2`).

In [ ]:
from obspy import UTCDateTime
nb_days = (UTCDateTime() - UTCDateTime("2011-03-11T05:46:23.200000Z"))/86400
print(round(nb_days))

Make a list of 10 UTCDateTime objects, starting today at 10:00 with a spacing of 90 minutes.

In [ ]:
times = []
current_time = UTCDateTime("2021-06-03T10:00:00.000000Z")
for i in range(0, 10):
    times.append(current_time + 90*60*i)
    
print(times)

Below is a list of strings with origin times of magnitude 8+ earthquakes since 2000 (fetched from IRIS). Assemble a list of inter-event times in days. Use matplotlib to display a histogram.

In [ ]:
times = ["2001-06-23T20:33:09",
         "2003-09-25T19:50:07",
         "2004-12-23T14:59:00",
         "2004-12-26T00:58:52",
         "2005-03-28T16:09:35",
         "2006-06-01T18:57:02",
         "2006-06-05T00:50:31",
         "2006-11-15T11:14:14",
         "2007-01-13T04:23:23",
         "2007-04-01T20:39:56",
         "2007-08-15T23:40:58",
         "2007-09-12T11:10:26",
         "2009-09-29T17:48:11",
         "2010-02-27T06:34:13",
         "2011-03-11T05:46:23",
         "2012-04-11T08:38:36",
         "2012-04-11T10:43:10",
         "2013-05-24T05:44:48"]

In [ ]:
import matplotlib.pyplot as plt

interevent = []
for i in range(0, len(times)-1):
    interevent.append((UTCDateTime(times[i+1])-UTCDateTime(times[i]))/86400)
print(interevent)

plt.hist(interevent, bins=range(0, 1000, 100))

### Trace Correction
 - Make an **`numpy.ndarray`** with zeros and (e.g. use **`numpy.zeros()`**) and put an ideal pulse somewhere in it
 - initialize a **`Trace`** object with your data array
 - Fill in some station information (e.g. network, station, ..)
 - Print trace summary and plot the trace
 - Change the sampling rate to 20 Hz
 - Change the starttime of the trace to the start time of this session
 - Print the trace summary and plot the trace again

In [ ]:
import numpy as np
from obspy import Trace, UTCDateTime

x = np.zeros(5000)
x[2500] = 1.0

tr = Trace(data=x)
tr.stats.sampling_rate = 20.0

tr.stats.network = 'IAG'
tr.stats.station = 'TUNGA'
tr.stats.channel = 'SHZ'

starttime = UTCDateTime()

tr.stats.starttime = starttime

print(tr)
tr.plot()


- Use **`tr.filter(...)`** and apply a lowpass filter with a corner frequency of 1 Hertz.
- Display the preview plot, there are a few seconds of zeros that we can cut off.

In [ ]:
tr.filter(type='lowpass', freq=1.0)
tr.plot()

- Use **`tr.trim(...)`** to remove some of the zeros at start and at the end
- show the preview plot again

In [ ]:
t1 = UTCDateTime("2021-06-17T04:03:00")
t2 = UTCDateTime("2021-06-17T04:03:15")
tr.trim(starttime=t1, endtime=t2)
tr.plot()

- Scale up the amplitudes of the trace by a factor of 500
- Add standard normal gaussian noise to the trace (use [**`np.random.randn()`**](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html))
- Display the preview plot again

In [ ]:
tr.data *= 500
tr.data = tr.data + np.random.randn(len(tr.data)) 

tr.plot()

### Stream Correction

- Read all Tohoku example earthquake data into a stream object ("./data/waveform\_\*")
- Print the stream summary

In [ ]:
from obspy import read

st = read("./data/waveform_*")
print(st)

- Use **`st.select()`** to only keep traces of station BFO in the stream. Show the preview plot.

In [ ]:
st = st.select(station='BFO')
print(st)
st.plot()

- trim the data to a 10 minute time window around the first arrival (just roughly looking at the preview plot)
- display the preview plot and spectrograms for the stream (with logarithmic frequency scale, use `wlen=50` for the spectrogram plot)

In [ ]:
from obspy import UTCDateTime

st.trim(starttime=UTCDateTime("2011-03-11T05:55:00"))

st.plot()

In [ ]:
st.spectrogram(log=True, wlen=50)

- remove the linear trend from the data, apply a tapering and a lowpass at 0.1 Hertz
- show the preview plot again

In [ ]:
st.detrend('linear')
st.taper(type="hann", max_percentage=0.05)
st.filter("lowpass", freq=0.1)
st.plot()

In [ ]:
st.spectrogram(log=True, wlen=50)